# Nézettségi adatok a top 5 bajnokságban 2000 és 2023 között

Programcsomagok importálása

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from functools import reduce
import seaborn as sns
import warnings

Webscraping a https://www.european-football-statistics.co.uk/ weboldalról

In [2]:
warnings.filterwarnings('ignore')

urls = ['https://www.european-football-statistics.co.uk/attn/nav/attnengleague.htm',
       'https://www.european-football-statistics.co.uk/attn/nav/attnesp.htm',
       'https://www.european-football-statistics.co.uk/attn/nav/attnita.htm',
       'https://www.european-football-statistics.co.uk/attn/nav/attnger.htm',
       'https://www.european-football-statistics.co.uk/attn/nav/attnfra.htm']

lst = []

for url in urls:
    if url == urls[0]:
        df = pd.read_html(url, encoding = 'ISO-8859-1')[4]
    else:
        df = pd.read_html(url, encoding = 'ISO-8859-1')[2]
        df = df.reset_index(drop=True)
    lst.append(df)

Adatokat tartalmazó lista szétbontása bajnokságonként

In [3]:
eng_df = lst[0]
spa_df = lst[1]
ita_df = lst[2]
ger_df = lst[3]
fra_df = lst[4]

eng_df.head(5)

,0,1,2,3,4,5,6,7
0,England,Premier,Champion,League 1,League 2,Best club,Average,Lg avg
1,2023,40.236,18.868,10.614,5.791,Manchester United,73.671,17.073
2,2022,39.600,16.760,10.009,4.915,Manchester United,73.150,15.991
3,2021,NaN,NaN,NaN,NaN,COVID,NaN,NaN
4,2020,39.349,18.585,8.753,4.664,Manchester United,73.393,15.998


Adattisztítás, évszámok és nézettségi adatok összeolvasztása

    -sajnos az egyes bajnoksághoz tartozó adatok struktúrája eltér, így minden országot külön kell kezelni

In [4]:
eng_df2 = eng_df.iloc[1:26,:2]
eng_df2 = eng_df2.drop(20)
eng_df2 = eng_df2.rename(columns={1 : 'eng'})

spa_df2 = spa_df.iloc[1:25, :2]
spa_df2 = spa_df2.rename(columns={1 : 'spa'})

ita_df2 = ita_df.iloc[1:25, :2]
ita_df2 = ita_df2.rename(columns={1 : 'ita'})

ger_df2 = ger_df.iloc[1:25, :2]
ger_df2 = ger_df2.rename(columns={1 : 'ger'})

fra_df2 = fra_df.iloc[1:25, :2]
fra_df2 = fra_df2.rename(columns={1 : 'fra'})

data_frames = [eng_df2, ita_df2, spa_df2, ger_df2, fra_df2]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on = 0,
                                            how='outer'), data_frames)

#df_merged.set_index(0, inplace = True)
df_merged.head(5)

,0,eng,ita,spa,ger,fra
0,2023,40.236,29.551,29.448,42.997,23.803
1,2022,39.600,NaN,NaN,NaN,NaN
2,2021,NaN,NaN,NaN,NaN,NaN
3,2020,39.349,27.469,28.864,40.882,22.526
4,2019,38.181,25.258,26.885,43.441,22.836


Adatok átalakítása számformátummá, üres sorok ejtése, ezres tagolás megszűntetése.

    -Probléma: 2021, 2022-ben a COVID19 miatt a bajnokságoban nem , vagy zárt kapuk mögött rendeztek mérkőzéseket, így nem állnak rendelkezésre nézettségi adatok.

        -Megoldási lehetőségek: 
        1. 2023-as adatok törlése, így 2000-2020 között teljes adatsor marad
        2. 2021-es és 2022-es adatok pótlása valamilyen statisztikai módszerrel

In [7]:
#df_merged2 = df_merged.drop(['2021','2022'])
df_merged2 = df_merged.astype(float).interpolate(method = 'linear')
df_merged2.rename(columns={0 : 'year'}, inplace = True)
df_merged2.iloc[:,1:] = df_merged2.iloc[:,1:] * 1000


df_merged2.to_csv(f'A top 5 bajnokság 2000-2023 közötti éves átlagos nézőszámadatai.csv', encoding = 'utf-8', index = False)

df_merged2.head(5)

,year,eng,ita,spa,ger,fra
0,2023.0,40236.0,29551.0,29448.000000,42997.0,23803.000000
1,2022.0,39600.0,28857.0,29253.333333,42292.0,23377.333333
2,2021.0,39474.5,28163.0,29058.666667,41587.0,22951.666667
3,2020.0,39349.0,27469.0,28864.000000,40882.0,22526.000000
4,2019.0,38181.0,25258.0,26885.000000,43441.0,22836.000000
